In [7]:
from dotenv import load_dotenv, find_dotenv
import pandas as pd
load_dotenv(find_dotenv())

True

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

In [10]:
import chardet

# Function to detect the encoding of a file
def detect_file_encoding(file_path):
    with open(file_path, 'rb') as file:
        # Read a portion of the file to detect encoding
        raw_data = file.read(50000)
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        return encoding

# Example usage
file_path = './data/Orders.csv'  # Replace with the actual file path
encoding = detect_file_encoding(file_path)
print(encoding)

Windows-1252


In [12]:
from tqdm import tqdm

def convert_csv_to_utf8(input_file_path, output_file_path, input_encoding):
    """
    Convert a CSV file from a specified encoding to UTF-8, with progress tracking.

    :param input_file_path: Path to the input CSV file.
    :param output_file_path: Path to the output CSV file.
    :param input_encoding: The encoding of the input file.
    """
    with open(input_file_path, 'r', encoding=input_encoding) as input_file:
        lines = input_file.readlines()

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for line in tqdm(lines, desc="Converting"):
            output_file.write(line)

# Example usage
input_file_path = './data/Orders.csv'  # Replace with your input file path
output_file_path = './data/Orders_utf8.csv'  # Replace with your output file path
input_encoding = 'Windows-1252'

convert_csv_to_utf8(input_file_path, output_file_path, encoding)

Converting: 100%|██████████| 9427/9427 [00:00<00:00, 793699.01it/s]


In [13]:
df = pd.read_csv('./data/Orders_utf8.csv')

In [15]:
df.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

In [16]:
df = df[['Row ID', 'Order Date', 'Unit Price', 'Customer ID']]

In [17]:
df.columns = ['id', 'created_at', 'subtotal_price', 'customer_id']

In [18]:
df

,id,created_at,subtotal_price,customer_id
0,18606,5/28/2012,2.88,2
1,20847,7/7/2010,2.84,3
2,23086,7/27/2011,6.68,3
3,23087,7/27/2011,5.68,3
4,23088,7/27/2011,205.99,3
...,...,...,...,...
9421,20275,5/14/2013,35.89,3402
9422,20276,5/14/2013,3.34,3402
9423,24491,9/12/2013,550.98,3402
9424,25914,2/8/2010,105.98,3403


In [24]:
df['created_at'] = pd.to_datetime(pd.to_datetime(df['created_at'], format='%m/%d/%Y'))

In [25]:
df.head()

,id,created_at,subtotal_price,customer_id
0,18606,2012-05-28,2.88,2
1,20847,2010-07-07,2.84,3
2,23086,2011-07-27,6.68,3
3,23087,2011-07-27,5.68,3
4,23088,2011-07-27,205.99,3


In [34]:
chat = ChatOpenAI(model="gpt-4", temperature=0.0)
agent = create_pandas_dataframe_agent(chat, df, verbose=True, handle_parsing_errors=True)

In [35]:
agent.run("What is the total revenue generated from all orders?")



> Entering new AgentExecutor chain...
Thought: The total revenue generated from all orders can be calculated by summing up the 'subtotal_price' column in the dataframe.
Action: python_repl_ast
Action Input: df['subtotal_price'].sum()
Observation: 832350.5399999999
Thought:I now know the final answer
Final Answer: The total revenue generated from all orders is approximately 832350.54.

> Finished chain.


'The total revenue generated from all orders is approximately 832350.54.'

In [36]:
agent.run("What is the average order value?")



> Entering new AgentExecutor chain...
Thought: To find the average order value, I need to calculate the mean of the 'subtotal_price' column in the dataframe.
Action: python_repl_ast
Action Input: df['subtotal_price'].mean()
Observation: 88.3036855506047
Thought:I now know the final answer
Final Answer: The average order value is approximately 88.30.

> Finished chain.


'The average order value is approximately 88.30.'

In [44]:
response = agent.run("What is the repeat order rate?")



> Entering new AgentExecutor chain...
Thought: The repeat order rate is the percentage of orders from customers who have made more than one purchase. To calculate this, we first need to find the total number of orders and the number of orders from customers who have made more than one purchase.

Action: python_repl_ast
Action Input: df['customer_id'].nunique()
Observation: 2703
Thought:The total number of unique customers is 2703. Now, I need to find the number of customers who have made more than one purchase.

Action: python_repl_ast
Action Input: df['customer_id'].value_counts().loc[lambda x : x>1].count()
Observation: 1806
Thought:The number of customers who have made more than one purchase is 1806. Now, I can calculate the repeat order rate by dividing the number of customers who have made more than one purchase by the total number of unique customers.

Action: python_repl_ast
Action Input: repeat_order_rate = 1806 / 2703
Observation: 
Thought:I have calculated the repeat order 

In [45]:
response

'The repeat order rate is approximately 66.81%.'

In [46]:
response_2 = agent.run("Calculate recency, frequency, and monetary value for each customer.")



> Entering new AgentExecutor chain...
Thought: To calculate recency, frequency, and monetary value for each customer, we need to perform the following steps:
1. Recency: Calculate the difference between the current date and the date of the last purchase for each customer.
2. Frequency: Count the number of purchases for each customer.
3. Monetary Value: Calculate the total amount of money spent by each customer.

Let's start with calculating the recency for each customer. We need to find the most recent purchase date for each customer and then calculate the difference from today's date. We can use the `groupby` function to group the data by `customer_id` and then use the `max` function to find the most recent date. After that, we can subtract this date from today's date to get the recency. 

Action: python_repl_ast
Action Input: df['created_at'] = pd.to_datetime(df['created_at'])
df['recency'] = (pd.Timestamp.today() - df.groupby('customer_id')['created_at'].transform('max')).dt.days


In [47]:
df

,id,created_at,subtotal_price,customer_id,recency,frequency,monetary_value
0,18606,2012-05-28,2.88,2,4232,1,2.88
1,20847,2010-07-07,2.84,3,3833,6,397.64
7,20228,2010-12-13,500.98,5,4248,2,507.46
9,24782,2011-05-26,90.24,6,4017,4,187.55
13,21866,2012-10-30,12.28,7,4077,1,12.28
...,...,...,...,...,...,...,...
9406,22599,2011-05-07,177.98,3398,4051,3,494.95
9409,26208,2010-03-29,11.97,3399,3700,4,106.86
9413,24911,2010-10-04,9.38,3400,3738,7,245.68
9420,18329,2011-04-12,212.60,3402,3760,4,802.81
